In [ ]:
!pip install "pillow == 11.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0


In [ ]:
# @title Install deps (run once)
!pip -q install "transformers>=4.45.0" accelerate bitsandbytes sentencepiece datasets --upgrade

import time, math, torch, gc, threading
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, set_seed
from threading import Thread

print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
PyTorch: 2.8.0+cu126
CUDA available: True


In [ ]:
#@title Imports, device, config
import time, statistics as stats, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# `infer_device` helper (use HF's if present)
try:
    from transformers import infer_device
except Exception:
    def infer_device():
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = infer_device()
print("Device:", device)

# ==== Config (edit freely) ====
MODEL_ID = "HuggingFaceTB/SmolLM-1.7B"
ASSIST_ID = "HuggingFaceTB/SmolLM-135M"

PROMPT = "Hugging Face is an open-source company that can be summarized as "
MAX_NEW_TOKENS = 60
NUM_ASSISTANT_TOKENS = 2          # try 2, 4, 8, 16
PROMPT_LOOKUP_NUM_TOKENS = 5      # as in your snippet
REPEATS = 5

# Precision: fp16 on CUDA (T4/A100/L4); fp32 on CPU
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32
print("dtype:", DTYPE)


Device: cuda
dtype: torch.float16


In [ ]:
#@title Load models/tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=DTYPE).to(device).eval()
assist = AutoModelForCausalLM.from_pretrained(ASSIST_ID, torch_dtype=DTYPE).to(device).eval()

# Silence "pad_token_id" messages by setting it on the configs
model.config.pad_token_id = model.config.eos_token_id
assist.config.pad_token_id = assist.config.eos_token_id

print("Loaded models.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loaded models.


In [ ]:
#@title Timing helpers (now also return decoded text)
@torch.inference_mode()
def time_generate_baseline(prompt: str, max_new_tokens: int):
    inputs = tok(prompt, return_tensors="pt").to(device)

    # tiny warmup
    _ = model.generate(**inputs, max_new_tokens=4, do_sample=False, use_cache=True,
                       pad_token_id=model.config.eos_token_id)

    if torch.cuda.is_available(): torch.cuda.synchronize()
    t0 = time.perf_counter()
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, use_cache=True,
                         pad_token_id=model.config.eos_token_id)
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t1 = time.perf_counter()

    total = t1 - t0
    gen_tokens = out.shape[1] - inputs["input_ids"].shape[1]
    tps = gen_tokens / total if total > 0 else float("nan")

    # --- NEW: qualitative outputs
    text_full = tok.decode(out[0], skip_special_tokens=True)
    # continuation only (slice off the prompt length)
    text_cont = tok.decode(out[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    return total, tps, gen_tokens, out, text_full, text_cont


@torch.inference_mode()
def time_generate_assisted(prompt: str, max_new_tokens: int,
                           num_assistant_tokens: int, prompt_lookup_num_tokens: int):
    inputs = tok(prompt, return_tensors="pt").to(device)

    # tiny warmup (assisted)
    _ = model.generate(**inputs, max_new_tokens=4, do_sample=False, use_cache=True,
                       assistant_model=assist, num_assistant_tokens=2,
                       pad_token_id=model.config.eos_token_id)

    kwargs = dict(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        use_cache=True,
        assistant_model=assist,
        num_assistant_tokens=num_assistant_tokens,
        prompt_lookup_num_tokens=prompt_lookup_num_tokens,
        pad_token_id=model.config.eos_token_id,
    )

    if torch.cuda.is_available(): torch.cuda.synchronize()
    t0 = time.perf_counter()
    out = model.generate(**kwargs)
    if torch.cuda.is_available(): torch.cuda.synchronize()
    t1 = time.perf_counter()

    total = t1 - t0
    gen_tokens = out.shape[1] - inputs["input_ids"].shape[1]
    tps = gen_tokens / total if total > 0 else float("nan")

    # --- NEW: qualitative outputs
    text_full = tok.decode(out[0], skip_special_tokens=True)
    text_cont = tok.decode(out[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    return total, tps, gen_tokens, out, text_full, text_cont


In [ ]:
#@title Show qualitative outputs (baseline vs assisted)
t_b, s_b, n_b, out_b, full_b, cont_b = time_generate_baseline(PROMPT, MAX_NEW_TOKENS)
t_a, s_a, n_a, out_a, full_a, cont_a = time_generate_assisted(
    PROMPT, MAX_NEW_TOKENS, NUM_ASSISTANT_TOKENS, PROMPT_LOOKUP_NUM_TOKENS
)

print("=== BASELINE continuation ===")
print(cont_b.strip())
print("\n=== ASSISTED continuation ===")
print(cont_a.strip())

same = (cont_b == cont_a)
print(f"\nSame continuation? {same}")
print(f"\nBaseline: {n_b} new toks in {t_b:.3f}s (TPS {n_b/max(t_b,1e-9):.2f})")
print(f"Assisted: {n_a} new toks in {t_a:.3f}s (TPS {n_a/max(t_a,1e-9):.2f})")


=== BASELINE continuation ===
100% free and open-source. It is a community-driven project that is based on the open-source software.
Hugging Face is a company that provides a platform for developers to build and deploy machine learning models. It is a company that provides a platform for developers to build

=== ASSISTED continuation ===
100% free and open-source. It is a community-driven project that is based on the open-source software.
Hugging Face is a company that provides a platform for developers to build and deploy machine learning models. It is a company that provides a platform for developers to build and deploy machine

Same continuation? False

Baseline: 60 new toks in 1.382s (TPS 43.40)
Assisted: 63 new toks in 1.125s (TPS 56.01)
